Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices
import random
from tqdm.notebook import tqdm
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm

import lab9_lib

In [2]:
INSTANCE = [1,2,5,10]       
GENOMES = 1000              #individual size
SIZE = 10                   #population size

In [3]:
def parent_selection(population, I):
    """choose a parent from the best I of the population"""
    parent = population[np.random.randint(0, I)]
    return parent

def xover(parent1, parent2):
    """crossover between the two parents"""
    return [p1 if r < .5 else p2 for p1, p2, r in zip(parent1, parent2, np.random.random(GENOMES))]

def mutate(ind):
    """new individual gaussian mutation"""
    SD = 0.2
    mutation = np.random.normal(0, SD, GENOMES)
    mutated = [int(round(c + m)) % 2 for c, m in zip(ind, mutation)]    #gaussian mutation
    # segments = [mutated[i:i+5] for i in range(0, len(mutated), 5)]      #divide the individual in segments
    # random.shuffle(segments)                                            #shuffle the segments
    # return [gen for seg in segments for gen in seg]                     #return the individual reassembled
    return mutated

In [4]:
FC = 0      #total number of fitness call

for I in INSTANCE:    
    fitness = lab9_lib.make_problem(I)
    
    individuals = []
    for k in range(SIZE):
        """initial population"""
        ind = choices([0,1], k=GENOMES)
        fit = fitness(ind)
        individuals.append((ind, fit))

    generations = 1
    no_improvement = 0
    best = 0.0

    individuals = sorted(individuals, key=lambda i:i[1], reverse=True)      #sorting based on fitness

    while True:
        generations += 1
        for k in range(SIZE//2):
            """substitute worst size/2 individuals with new ones"""
            p1, p2 = parent_selection(individuals, SIZE/2), parent_selection(individuals, SIZE/2)
            newInd = mutate(xover(p1[0], p2[0]))
            individuals[SIZE-1-k] = (newInd, fitness(newInd))
        individuals = sorted(individuals, key=lambda i:i[1], reverse=True)  #sorting based on fitness
        if (individuals[0][1] - best > 0):
            if (individuals[0][1] - best)*100 > 0.1:
                no_improvement = 0      #fitness improvement good enough: let's keep going with the algorithm
            best = individuals[0][1]
        no_improvement += 1
        if no_improvement == 2_000: break   #for n times the fitness improvement has not been good enough so stop the algorithm

    FC += fitness.calls     #increment total number of fitness call

    print(f"Problem instance {I}")
    print(f"Best fitness: {individuals[0][1]:.2%}")
    print(f"Fitness calls: {fitness.calls}")
    print(f"Number of generations: {generations}")
    print("")

print(f"Total number of fitness calls: {FC}")

Problem instance 1
Best fitness: 97.00%
Fitness calls: 98745
Number of generations: 19748

Problem instance 2
Best fitness: 47.80%
Fitness calls: 28530
Number of generations: 5705

Problem instance 5
Best fitness: 48.88%
Fitness calls: 68720
Number of generations: 13743

Problem instance 10
Best fitness: 31.99%
Fitness calls: 70730
Number of generations: 14145

Total number of fitness calls: 266725
